<a href="https://colab.research.google.com/github/heliojunior1/scripts_b3/blob/main/oplab_opcoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests python-dotenv

In [ ]:
import requests
from google.colab import userdata # For Colab secrets
from datetime import datetime, timedelta

def carregar_credenciais_colab():
    """Loads OPLAB email and password from Colab secrets."""
    email = userdata.get("OPLAB_EMAIL")
    senha = userdata.get("OPLAB_SENHA")
    if not email or not senha:
        raise ValueError("Credenciais 'OPLAB_EMAIL' ou 'OPLAB_SENHA' não encontradas nos Secrets do Colab. "
                         "Por favor, configure-as conforme as instruções acima (clique no ícone de chave 🔑 na barra lateral).")
    return email, senha

def autenticar(email, senha):
    url = "https://api.oplab.com.br/v3/domain/users/authenticate"
    payload = {"email": email, "password": senha}
    resp = requests.post(url, json=payload)
    if resp.status_code != 200:
        raise Exception(f"Erro na autenticação: {resp.status_code} - {resp.text}")
    token = resp.json().get("access-token")
    if not token:
        raise Exception("Access token não encontrado na resposta da autenticação.")
    return token

def obter_opcoes(token, ativo):
    url = f"https://api.oplab.com.br/v3/market/options/{ativo}"
    headers = {"Access-Token": token}
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        raise Exception(f"Erro ao obter opções: {resp.status_code} - {resp.text}")
    data = resp.json()
    if not isinstance(data, list):
        raise Exception(f"Resposta inesperada ao obter opções: {data}")
    return data

def converter_float(valor):
    try:
        return float(valor)
    except (ValueError, TypeError):
        return None

def filtrar_opcoes(opcoes, config):
    filtradas = []
    for opcao in opcoes:
        if not opcao.get("due_date"):
            continue
        try:
            vencimento = datetime.strptime(opcao["due_date"], "%Y-%m-%d")
        except Exception:
            continue
        if vencimento > config["vencimento_maximo"]:
            continue
        if opcao.get("type", "").upper() != config["tipo_opcao_filtro"].upper():
            continue
        if not opcao.get("last_trade_at"): # Check if last_trade_at exists
            continue

        ultimo_preco = converter_float(opcao.get("close"))
        preco_venda = converter_float(opcao.get("ask"))
        quantidade = converter_float(opcao.get("ask_volume"))

        if None in (ultimo_preco, preco_venda, quantidade):
            continue
        if 0.0 in (ultimo_preco, preco_venda, quantidade): # Explicitly check for 0.0
            continue

        try:
            # OPLab API returns timestamp in milliseconds
            last_trade = datetime.fromtimestamp(opcao["last_trade_at"] / 1000)
        except Exception:
            continue

        if last_trade < datetime.now() - timedelta(days=config["days_ignorar"]):
            continue

        limite_superior = ultimo_preco * (1 + config["percentual"] / 100)
        valor_total = preco_venda * quantidade
        pct_venda_ultimo = ((preco_venda / ultimo_preco) - 1) * 100 if ultimo_preco > 0 else 0

        if preco_venda <= limite_superior and valor_total >= config["valor_minimo_negociavel"]:
            filtradas.append({
                "nome": opcao["symbol"],
                "tipo": opcao["type"],
                "vencimento": opcao["due_date"],
                "ultimo_preco": ultimo_preco,
                "preco_venda": preco_venda,
                "quantidade_disponivel": quantidade,
                "valor_total_negociavel": valor_total,
                "percentual_venda_sobre_ultimo": pct_venda_ultimo,
                "ultima_negociacao": last_trade.strftime("%Y-%m-%d %H:%M:%S")
            })
    return filtradas

def exibir_opcoes(opcoes, ativo, percentual, valor_minimo):
    if not opcoes:
        print(f"Não foram encontradas opções de {ativo} que atendam aos critérios especificados (incluindo valor total negociável mínimo de R$ {valor_minimo:.2f}).")
        return
    opcoes.sort(key=lambda x: x["valor_total_negociavel"], reverse=True)
    print(f"\nOpções de {ativo} com preço de venda até {percentual}% acima do último preço negociado, preços não nulos e valor total negociável mínimo de R$ {valor_minimo:.2f}:")
    for o in opcoes:
        print(
            f"{o['nome']} - {o['tipo']}- {o['vencimento']} - "
            f"LAST:{o['ultimo_preco']:.2f} -  ASK:{o['preco_venda']:.2f} - "
            f"PCT:{o['percentual_venda_sobre_ultimo']:.2f}% - "
            f"DTHLAST: {o['ultima_negociacao']} -  "
            f"QuantD: {o['quantidade_disponivel']:.0f}, " # Format quantity as integer
            f"VTD: R$ {o['valor_total_negociavel']:.2f}"
        )


In [ ]:
def run_analysis():
    # Configuration for the analysis - adjust as needed
    config = {
        "ativo": "VALE3",  # Asset to analyze (e.g., "PETR4", "VALE3")
        "percentual": 30,  # Max percentage above last price for sell price (e.g., 30 for 30%)
        "valor_minimo_negociavel": 600,  # Minimum negotiable total value (e.g., 100 for R$100)
        "tipo_opcao_filtro": "PUT",  # Type of option: "CALL" or "PUT"
        "days_ignorar": 4,  # Ignore options not traded in the last X days
        "vencimento_maximo": datetime.strptime("2025-12-20", "%Y-%m-%d") # Max expiration date (YYYY-MM-DD)
    }
    try:
        print("Carregando credenciais do Colab Secrets...")
        email, senha = carregar_credenciais_colab() # Use the Colab-specific function

        print("Autenticando com a API OPLab...")
        token = autenticar(email, senha)
        print("Autenticação bem-sucedida.")

        print(f"Obtendo opções para o ativo {config['ativo']}...")
        opcoes = obter_opcoes(token, config["ativo"])
        print(f"Total de {len(opcoes)} opções recebidas da API.")

        print("Filtrando opções com base nos critérios definidos...")
        filtradas = filtrar_opcoes(opcoes, config)
        print(f"{len(filtradas)} opções atendem aos critérios após a filtragem.")

        exibir_opcoes(filtradas, config["ativo"], config["percentual"], config["valor_minimo_negociavel"])

    except ValueError as ve: # Specific handling for credential errors from carregar_credenciais_colab
        print(f"Erro de Configuração: {ve}")
    except Exception as e:
        print(f"Ocorreu um erro durante a execução: {e}")

# Run the analysis
if __name__ == '__main__': # Standard way to run main logic, works well in Colab too
    run_analysis()

Carregando credenciais do Colab Secrets...
Autenticando com a API OPLab...
Autenticação bem-sucedida.
Obtendo opções para o ativo VALE3...
Total de 2286 opções recebidas da API.
Filtrando opções com base nos critérios definidos...
3 opções atendem aos critérios após a filtragem.

Opções de VALE3 com preço de venda até 30% acima do último preço negociado, preços não nulos e valor total negociável mínimo de R$ 600.00:
VALET570W1 - PUT- 2025-08-01 - LAST:1.66 -  ASK:2.00 - PCT:20.48% - DTHLAST: 2025-07-28 09:05:13 -  QuantD: 2900, VTD: R$ 5800.00
VALEU588 - PUT- 2025-09-19 - LAST:3.18 -  ASK:3.88 - PCT:22.01% - DTHLAST: 2025-07-28 09:05:11 -  QuantD: 1000, VTD: R$ 3880.00
VALEX600 - PUT- 2025-12-19 - LAST:2.81 -  ASK:2.82 - PCT:0.36% - DTHLAST: 2025-07-28 09:05:11 -  QuantD: 500, VTD: R$ 1410.00
